In [1]:
import sys
sys.path.insert(0, "../../")
from pyecharts import online
online()
#  演示必要的准备代码，使用该库时不需重复此单元格命令

In [2]:
import xalpha as xa
import pandas as pd

# 策略制定类
生成简单的策略投资交易单，并储存在 self.status 中

### 一次性买入并持有

In [3]:
jshs = xa.fundinfo('000311')
# 选择投资标的

In [13]:
bah = xa.policy.buyandhold(jshs, start='2017-01-01', totmoney=100000)
# 简单的一次性买入类，从 start 日买入后就一直持有，始终选择分红再投入

In [5]:
bah.status
# 第二三列表示此两日恰为分红日，选择了分红再投入

,000311,date
0,100000.00,2017-01-03
1,0.05,2017-08-04
2,0.05,2017-08-15


In [6]:
jshs.fenhongdate
# 验证该两日确实分红

[Timestamp('2017-08-04 00:00:00'), Timestamp('2017-08-15 00:00:00')]

In [14]:
bah.sellout('2018-06-01') # 选定日期全部卖出

In [15]:
bah.status

,000311,date
0,100000.000,2017-01-03
1,0.050,2017-08-04
2,0.050,2017-08-15
3,-0.005,2018-06-01


In [16]:
jshstrade = xa.trade(jshs, bah.status) # 尝试交易一下

In [20]:
jshstrade.xirrrate()

0.15629726951248488

### 定投类

In [35]:
# 依旧先制定标的
zzcm = xa.fundinfo('164818')
zzcm

工银中证传媒指数分级

In [36]:
auto = xa.policy.scheduled(zzcm, 1000, pd.date_range('2015-07-01','2018-07-01',freq='W-THU'))
# 按照每周四定额定投1000，生成交易单

In [38]:
# 按上面的策略单子交易下
cm_t3 = xa.trade(zzcm, auto.status)

In [40]:
cm_t3.v_tradevolume() # 交易情况可视化，那些明显的缝隙对应了小长假等，当次定投下单会被自动延后到下一个交易日

In [41]:
cm_t3.dailyreport('2018-08-03')  ## 截止到2018.08.03， 定投已经浮亏了 40% 多

{'currentshare': 96030.82,
 'currentvalue': 89788.82,
 'date': Timestamp('2018-08-03 00:00:00'),
 'originalvalue': 155000.0,
 'returnrate': -42.0717,
 'unitcost': 1.6141,
 'unitvalue': 0.93500000000000005}

In [42]:
cm_t3.dailyreport('2018-08-03')['returnrate']

-42.0717

还可以根据指数位置设置定期不定额的定投策略生成，这里就不仔细展开，只给一个例子：

In [47]:
auto2 = xa.policy.scheduled_tune(zzcm, 1000, pd.date_range('2015-07-01','2018-07-01',freq='M'),
                                [(0.9,2),(1.2,1)]) # 净值0.9以下加倍定投，1.2以上不再定投，中间正常定投

In [50]:
cm_t4 = xa.trade(zzcm, auto2.status)

In [51]:
cm_t4.v_tradevolume()

### 网格类

In [3]:
gfc = xa.fundinfo('002903') 
gfc
# 场外网格最好选择基金的C份额

广发中证500ETF联接C

In [5]:
gr = xa.policy.grid(gfc,[3,3,3,3,3,3],[6,6,6,6,6,6],'2017-01-01', '2018-08-03')
# 制定一个 17年开始，18年8月结束的，以开始日期价格为基准，每下跌3%买入一仓，每仓买入上涨6%后卖出的网格策略，一共设置6档
## 注意网格对应的总金额可通过 totmoney= 参数设置，这里默认值是100000
## 卖出实际并不严格对应每次买入的份额，而是买入均分金额分仓，卖出均分持有份额分仓

In [18]:
gr.sellpts,gr.buypts # 网格计划对应的卖点和买点净值

([1.09791196,
  1.0649746012000001,
  1.0330253631639998,
  1.0020346022690798,
  0.97197356420100745,
  0.9428143572749772],
 [1.035766,
  1.0046930199999999,
  0.97455222939999986,
  0.94531566251799981,
  0.91695619264245976,
  0.88944750686318597])

In [22]:
gfc.price.iloc[-3:-1] # 事实上中证500已大幅跌破网格底线

,comment,date,netvalue,totvalue
522,0,2018-08-03,0.8585,0.8585
523,0,2018-08-06,0.8407,0.8407


In [6]:
gfc_t = xa.trade(gfc, gr.status) #  请开始你的交易

In [12]:
gfc_t.v_tradevolume(bar_category_gap="90%") # 交易情况可视化

In [13]:
gfc_t.xirrrate('2018-04-01') # 截止指定日期的年化收益率超过50%

0.51106565276944

In [15]:
gfc_t.xirrrate() # 随着18年六七月份市场大跌，网格被跌穿，收益年化转负

-0.0727373855329021

In [17]:
gfc_t.v_totvalue() # 网格投资持有基金的总值情况

In [23]:
# 考虑到网格的闲置资金可以在货基中吃收益，于是使用投资组合分析
gfc_cb = xa.mulfix(gfc_t) # 恰好 mulfix 类的默认 totmoney 也是 100000，我们无需在额外设置

In [24]:
gfc_cb.combsummary() # 由于程序逻辑是以历史累计投入作为分母计算收益率，因此对于这种“高频”交易，收益率被拉低
# 实际系统，即使截止8月初，收益率在 2% 左右

,基金名称,基金代码,基金现值,基金总申购,基金持有成本,基金了结收入,基金收益总额,投资收益率
0,广发中证500ETF联接C,002903,91735.16,183333.33,94193.01,89140.32,-2457.85,-1.3406
1,货币基金,mf,10398.03,172473.65,5806.99,166666.66,4591.04,2.6619
2,总计,xxxxxx,102133.19,355806.98,100000.00,255806.98,2133.19,0.5995


In [25]:
gfc_cb.xirrrate() # 系统总计收益年化勉强为正

0.013656306786996857